In [1]:
import itertools
from parlay_system import *
pd.set_option('display.max_rows', 500)


In [2]:
# Dec 15 games
monk = MoneyLine(event="monk", bet_amount=100, odds=-182)
justicia = MoneyLine(event="justicia", bet_amount=100, odds=140)

boulton = MoneyLine(event="boulton", bet_amount=100, odds=-155)
baggish = MoneyLine(event="baggish", bet_amount=100, odds=124)

richardson = MoneyLine(event="richardson", bet_amount=100, odds=-560)
suzuki = MoneyLine(event="suzuki", bet_amount=100, odds=380)

smith = MoneyLine(event="smith", bet_amount=100, odds=-500)
woodhouse = MoneyLine(event="woodhouse", bet_amount=100, odds=350)

lennon = MoneyLine(event="lennon", bet_amount=100, odds=-167)
rydz = MoneyLine(event="rydz", bet_amount=100, odds=135)

oconnor = MoneyLine(event="oconnor", bet_amount=100, odds=-385)
kantele = MoneyLine(event="kantele", bet_amount=100, odds=290)

binaries = [
    [monk, justicia],
    [boulton, baggish],
    [richardson, suzuki],
    [smith, woodhouse],
    [lennon, rydz],
    [oconnor, kantele]
]


results = [
    [0, 1], #[monk, justicia],
    [0, 1], #[boulton, baggish],
    [0, 1], #[richardson, suzuki],
    [0, 1], #[smith, woodhouse],
    [1, 0], #[lennon, rydz],
    [1, 0], #[oconnor, kantele]
]

def create_dicts(in_binaries, in_results):
    ml_lookup = {}
    ml_win_lookup = {}
    id = 0
    for bi in range(len(in_binaries)):
        binary = in_binaries[bi]
        for ml_i in range(len(binary)):
            ml = binary[ml_i]
            ml.set_index(id) 
            is_winner = in_results[bi][ml_i]
            ml_lookup[id] = ml.event
            ml_win_lookup[id] = is_winner
            id += 1
    
    return ml_lookup, ml_win_lookup

ml_lookup, ml_outcome_lookup = create_dicts(binaries, results)



In [3]:
#ml_lookup

In [4]:
#ml_outcome_lookup

In [5]:
def findsubsets(s, n):
    return list(itertools.combinations(s, n))

TOURNEY_SIZE = len(binaries)
SELECT_NUM = 3
s = set()
n = SELECT_NUM

for i in range(TOURNEY_SIZE):
    s.add(i)

all_subsets = findsubsets(s, n)

In [6]:
len(all_subsets)

20

In [7]:
csv = []
df = pd.DataFrame()

for i in range(len(all_subsets)):
    current_subset = all_subsets[i]
    parlay_binaries = []
    for i in range(len(current_subset)):
        index = current_subset[i]
        parlay_binaries.append(binaries[index])

    ps = ParlaySystem(binaries=parlay_binaries,
                    target_profit=500,
                    bounds=(0.01, 30),
                    binary_index_arr=list(current_subset),
                    binary_results_arr=results,
                    index_to_ml=ml_lookup,
                    index_to_outcome=ml_outcome_lookup
                    )
    csv_res, raw_df = ps.slsqp_solver()
    df = pd.concat([df, raw_df])
    csv.append(csv_res)

     fun: -4.567398604363823
     jac: array([ 0.49946356, -0.65445763,  0.36368138, -1.20746154,  0.29331797,
       -1.49403226,  0.083     , -2.3506    ])
 message: 'Positive directional derivative for linesearch'
    nfev: 229
     nit: 18
    njev: 14
  status: 8
 success: False
       x: array([0.01      , 0.75609785, 0.27260826, 0.82057758, 0.36046586,
       0.8421497 , 0.53000656, 0.88377807])
slsqp_solver: 
                          event    index[]     result  event_status   bet  \
7      justicia_baggish_suzuki_  [1, 3, 5]  [1, 1, 1]          True  0.88   
5      justicia_boulton_suzuki_  [1, 2, 5]  [1, 0, 1]         False  0.84   
3          monk_baggish_suzuki_  [0, 3, 5]  [0, 1, 1]         False  0.82   
1          monk_boulton_suzuki_  [0, 2, 5]  [0, 0, 1]         False  0.76   
6  justicia_baggish_richardson_  [1, 3, 4]  [1, 1, 0]         False  0.53   
4  justicia_boulton_richardson_  [1, 2, 4]  [1, 0, 0]         False  0.36   
2      monk_baggish_richardson_  [0, 3, 

slsqp_solver: 
                         event    index[]     result  event_status   bet  \
6      justicia_suzuki_lennon_  [1, 5, 8]  [1, 1, 1]          True  0.84   
7        justicia_suzuki_rydz_  [1, 5, 9]  [1, 1, 0]         False  0.89   
3            monk_suzuki_rydz_  [0, 5, 9]  [0, 1, 0]         False  0.83   
2          monk_suzuki_lennon_  [0, 5, 8]  [0, 1, 1]         False  0.76   
5    justicia_richardson_rydz_  [1, 4, 9]  [1, 0, 0]         False  0.56   
4  justicia_richardson_lennon_  [1, 4, 8]  [1, 0, 1]         False  0.36   
1        monk_richardson_rydz_  [0, 4, 9]  [0, 0, 0]         False  0.33   
0      monk_richardson_lennon_  [0, 4, 8]  [0, 0, 1]         False  0.01   

        mult   payout  profit  
6  18.418204  15.5117   10.93  
7  27.072000  24.1565   19.57  
3  17.477802  14.5723    9.99  
2  11.890873   8.9911    4.41  
5   6.647143   3.7528   -0.83  
4   4.522327   1.6302   -2.96  
1   4.291425   1.3996   -3.19  
0   2.919634   0.0292   -4.56  
total_bet   

                        event    index[]     result  event_status   bet  \
6      baggish_suzuki_lennon_  [3, 5, 8]  [1, 1, 1]          True  0.82   
7        baggish_suzuki_rydz_  [3, 5, 9]  [1, 1, 0]         False  0.88   
3        boulton_suzuki_rydz_  [2, 5, 9]  [0, 1, 0]         False  0.83   
2      boulton_suzuki_lennon_  [2, 5, 8]  [0, 1, 1]         False  0.76   
5    baggish_richardson_rydz_  [3, 4, 9]  [1, 0, 0]         False  0.50   
1    boulton_richardson_rydz_  [2, 4, 9]  [0, 0, 0]         False  0.33   
4  baggish_richardson_lennon_  [3, 4, 8]  [1, 0, 1]         False  0.27   
0  boulton_richardson_lennon_  [2, 4, 8]  [0, 0, 1]         False  0.01   

        mult   payout  profit  
6  17.190323  14.1066    9.70  
7  25.267200  22.1750   17.77  
3  18.557419  15.4722   11.07  
2  12.625381   9.5464    5.14  
5   6.204000   3.1318   -1.27  
1   4.556509   1.4860   -2.92  
4   4.220838   1.1507   -3.25  
0   3.099982   0.0310   -4.37  
total_bet   :  4.4
mean        :  3.

slsqp_solver: 
                           event     index[]     result  event_status   bet  \
6      suzuki_woodhouse_oconnor_  [5, 7, 10]  [1, 1, 1]          True  0.93   
7      suzuki_woodhouse_kantele_  [5, 7, 11]  [1, 1, 0]         False  0.98   
5          suzuki_smith_kantele_  [5, 6, 11]  [1, 0, 0]         False  0.92   
3  richardson_woodhouse_kantele_  [4, 7, 11]  [0, 1, 0]         False  0.91   
4          suzuki_smith_oconnor_  [5, 6, 10]  [1, 0, 1]         False  0.76   
2  richardson_woodhouse_oconnor_  [4, 7, 10]  [0, 1, 1]         False  0.74   
1      richardson_smith_kantele_  [4, 6, 11]  [0, 0, 0]         False  0.68   
0      richardson_smith_oconnor_  [4, 6, 10]  [0, 0, 1]         False  0.01   

        mult   payout  profit  
6  27.210390  25.4312   19.50  
7  84.240000  82.4263   76.50  
5  22.464000  20.6877   14.76  
3  20.683929  18.9087   12.98  
4   7.256104   5.4890   -0.44  
2   6.681122   4.9143   -1.02  
1   5.515714   3.7496   -2.18  
0   1.781633   0.

In [8]:
df.describe()

,bet,mult,payout,profit
count,160.000000,160.000000,160.000000,160.000000
mean,0.600812,13.295820,10.571109,5.764062
std,0.303582,12.394402,12.525693,12.362279
min,0.010000,1.781633,0.017800,-5.910000
25%,0.352500,4.600158,1.512800,-2.872500
50%,0.740000,9.422637,6.911800,1.935000
75%,0.830000,16.454003,13.488950,8.972500
max,0.980000,84.240000,82.426300,76.500000


In [9]:
negative_df = df.loc[df.profit < 0]
negative_df = negative_df.sort_values(by=['event_status', 'profit'], ascending=[False, True])
negative_df.describe()

,bet,mult,payout,profit
count,67.000000,67.000000,67.000000,67.000000
mean,0.300746,4.481913,1.687391,-3.028806
std,0.229923,1.647832,1.630738,1.666029
min,0.010000,1.781633,0.017800,-5.910000
25%,0.010000,3.162173,0.031850,-4.325000
50%,0.330000,4.297581,1.284800,-3.170000
75%,0.430000,6.097188,2.703300,-1.280000
max,0.760000,7.561866,5.489000,-0.030000


In [10]:
positive_df = df.loc[df.profit >= 0]
positive_df = positive_df.sort_values(by=['event_status', 'profit'], ascending=[False, False])
positive_df.describe()

,bet,mult,payout,profit
count,93.000000,93.000000,93.000000,93.000000
mean,0.816989,19.645624,16.971206,12.098710
std,0.092428,12.892493,13.052322,12.855457
min,0.500000,7.699355,4.115700,0.110000
25%,0.760000,11.031429,8.445000,3.440000
50%,0.820000,14.512208,11.850600,6.910000
75%,0.880000,25.267200,22.175000,17.770000
max,0.980000,84.240000,82.426300,76.500000


In [11]:
negative_df.head(200)

,event,index[],result,event_status,bet,mult,payout,profit
4,baggish_lennon_oconnor_,"[3, 8, 10]","[1, 1, 1]",True,0.27,4.511530,1.2299,-2.95
4,justicia_lennon_oconnor_,"[1, 8, 10]","[1, 1, 1]",True,0.36,4.833782,1.7425,-2.63
6,justicia_baggish_oconnor_,"[1, 3, 10]","[1, 1, 1]",True,0.53,6.772364,3.5893,-0.67
0,richardson_smith_oconnor_,"[4, 6, 10]","[0, 0, 1]",False,0.01,1.781633,0.0178,-5.91
0,monk_richardson_smith_,"[0, 4, 6]","[0, 0, 0]",False,0.01,2.191366,0.0219,-5.40
0,richardson_smith_lennon_,"[4, 6, 8]","[0, 0, 1]",False,0.01,2.261163,0.0226,-5.35
0,monk_richardson_oconnor_,"[0, 4, 10]","[0, 0, 1]",False,0.01,2.300460,0.0230,-5.29
0,boulton_richardson_smith_,"[2, 4, 6]","[0, 0, 0]",False,0.01,2.326728,0.0233,-5.26
0,monk_smith_oconnor_,"[0, 6, 10]","[0, 0, 1]",False,0.01,2.342286,0.0234,-5.24
0,richardson_lennon_oconnor_,"[4, 8, 10]","[0, 1, 1]",False,0.01,2.373732,0.0237,-5.23


In [12]:
positive_df.head(200)

,event,index[],result,event_status,bet,mult,payout,profit
7,justicia_suzuki_woodhouse_,"[1, 5, 7]","[1, 1, 1]",True,0.96,51.840000,49.6308,44.21
7,baggish_suzuki_woodhouse_,"[3, 5, 7]","[1, 1, 1]",True,0.95,48.384000,46.0432,40.76
6,suzuki_woodhouse_lennon_,"[5, 7, 8]","[1, 1, 1]",True,0.93,34.534132,32.2707,26.90
6,suzuki_woodhouse_oconnor_,"[5, 7, 10]","[1, 1, 1]",True,0.93,27.210390,25.4312,19.50
7,justicia_baggish_suzuki_,"[1, 3, 5]","[1, 1, 1]",True,0.88,25.804800,22.8057,18.33
7,justicia_baggish_woodhouse_,"[1, 3, 7]","[1, 1, 1]",True,0.87,24.192000,21.1404,16.72
6,justicia_suzuki_lennon_,"[1, 5, 8]","[1, 1, 1]",True,0.84,18.418204,15.5117,10.93
6,justicia_woodhouse_lennon_,"[1, 7, 8]","[1, 1, 1]",True,0.83,17.267066,14.3091,9.78
6,baggish_suzuki_lennon_,"[3, 5, 8]","[1, 1, 1]",True,0.82,17.190323,14.1066,9.70
6,baggish_woodhouse_lennon_,"[3, 7, 8]","[1, 1, 1]",True,0.81,16.115928,12.9774,8.64


In [13]:
positive_df.groupby('event_status').count()

,event,index[],result,bet,mult,payout,profit
event_status,,,,,,,
False,76,76,76,76,76,76,76
True,17,17,17,17,17,17,17


In [14]:
negative_df.groupby('event_status').count()

,event,index[],result,bet,mult,payout,profit
event_status,,,,,,,
False,64,64,64,64,64,64,64
True,3,3,3,3,3,3,3


In [15]:
neg = negative_df.reset_index(drop=True).style.applymap(color_negative_red)
neg

,event,index[],result,event_status,bet,mult,payout,profit
0,baggish_lennon_oconnor_,"['3', '8', '10']","[1, 1, 1]",True,0.27,4.51153,1.2299,-2.95
1,justicia_lennon_oconnor_,"['1', '8', '10']","[1, 1, 1]",True,0.36,4.83378,1.7425,-2.63
2,justicia_baggish_oconnor_,"['1', '3', '10']","[1, 1, 1]",True,0.53,6.77236,3.5893,-0.67
3,richardson_smith_oconnor_,"['4', '6', '10']","[0, 0, 1]",False,0.01,1.78163,0.0178,-5.91
4,monk_richardson_smith_,"['0', '4', '6']","[0, 0, 0]",False,0.01,2.19137,0.0219,-5.4
5,richardson_smith_lennon_,"['4', '6', '8']","[0, 0, 1]",False,0.01,2.26116,0.0226,-5.35
6,monk_richardson_oconnor_,"['0', '4', '10']","[0, 0, 1]",False,0.01,2.30046,0.023,-5.29
7,boulton_richardson_smith_,"['2', '4', '6']","[0, 0, 0]",False,0.01,2.32673,0.0233,-5.26
8,monk_smith_oconnor_,"['0', '6', '10']","[0, 0, 1]",False,0.01,2.34229,0.0234,-5.24
9,richardson_lennon_oconnor_,"['4', '8', '10']","[0, 1, 1]",False,0.01,2.37373,0.0237,-5.23


In [16]:
pos = positive_df.reset_index(drop=True).style.applymap(color_positive_green)
pos

,event,index[],result,event_status,bet,mult,payout,profit
0,justicia_suzuki_woodhouse_,"['1', '5', '7']","[1, 1, 1]",True,0.96,51.84,49.6308,44.21
1,baggish_suzuki_woodhouse_,"['3', '5', '7']","[1, 1, 1]",True,0.95,48.384,46.0432,40.76
2,suzuki_woodhouse_lennon_,"['5', '7', '8']","[1, 1, 1]",True,0.93,34.5341,32.2707,26.9
3,suzuki_woodhouse_oconnor_,"['5', '7', '10']","[1, 1, 1]",True,0.93,27.2104,25.4312,19.5
4,justicia_baggish_suzuki_,"['1', '3', '5']","[1, 1, 1]",True,0.88,25.8048,22.8057,18.33
5,justicia_baggish_woodhouse_,"['1', '3', '7']","[1, 1, 1]",True,0.87,24.192,21.1404,16.72
6,justicia_suzuki_lennon_,"['1', '5', '8']","[1, 1, 1]",True,0.84,18.4182,15.5117,10.93
7,justicia_woodhouse_lennon_,"['1', '7', '8']","[1, 1, 1]",True,0.83,17.2671,14.3091,9.78
8,baggish_suzuki_lennon_,"['3', '5', '8']","[1, 1, 1]",True,0.82,17.1903,14.1066,9.7
9,baggish_woodhouse_lennon_,"['3', '7', '8']","[1, 1, 1]",True,0.81,16.1159,12.9774,8.64
